## Préprocessing

In [22]:
import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer

In [23]:
df = pd.read_excel('gamelist4.xlsx')

In [24]:
df.head()

,Unnamed: 0,gameId,Name,Price,Genres,Rating,hasDLC,inAppPurchases,Release Date,Multiplayer
0,0,1903340,Clair Obscur: Expedition 33,"67,48€","['Action', 'RPG']",Very Positive,True,False,2025-04-24,False
1,1,2993780,FANTASY LIFE i: The Girl Who Steals Time,"59,99€",['RPG'],Very Positive,False,False,2025-05-21,True
2,2,730,Counter-Strike 2,Free To Play,['Action'],Very Positive,True,True,2012-08-21,True
3,3,1091500,Cyberpunk 2077,"59,99€",['RPG'],Very Positive,True,False,2020-12-09,False
4,4,2488370,Cash Cleaner Simulator,"14,79€","['Casual', 'Indie', 'Simulation']",Very Positive,True,False,2025-05-08,False


In [25]:
# Drop useless column
df = df.drop(columns='Unnamed: 0')

In [26]:
# Create a column for each genre
df['Genres'] = df['Genres'].apply(lambda x: ast.literal_eval(x))
mlb = MultiLabelBinarizer()
genres_encoded = mlb.fit_transform(df['Genres'])
genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_)
df = pd.concat([df, genres_df], axis=1)
df = df.drop(columns='Genres')

In [27]:
# Transform into binary variables
df['Multiplayer'] = df['Multiplayer'].transform(lambda x : 0 if x == False else 1)
df['inAppPurchases'] = df['inAppPurchases'].transform(lambda x : 0 if x == False else 1)
df['hasDLC'] = df['hasDLC'].transform(lambda x : 0 if x == False else 1)

In [28]:
# Transform prices into float
df['Price'] = df['Price'].transform(lambda x : '0,00€' if x == 'Free To Play' or x == 'Free to Play' else x)
df['Price'] = df['Price'].str.replace('--', '00', regex=False)
df = df[df['Price'].apply(lambda x: isinstance(x, str) and x.endswith('€') and ',' in x)]
df['Price'] 
df['Price'] = (
    df['Price']
    .str[:-1]
    .str.replace(',', '.')
    .str.replace('€', '')
    .astype(float)
)

In [29]:
df.head()

,gameId,Name,Price,Rating,hasDLC,inAppPurchases,Release Date,Multiplayer,Action,Adventure,Casual,Early Access,Indie,Massively Multiplayer,RPG,Racing,Simulation,Sports,Strategy
0,1903340,Clair Obscur: Expedition 33,67.48,Very Positive,1,0,2025-04-24,0,1,0,0,0,0,0,1,0,0,0,0
1,2993780,FANTASY LIFE i: The Girl Who Steals Time,59.99,Very Positive,0,0,2025-05-21,1,0,0,0,0,0,0,1,0,0,0,0
2,730,Counter-Strike 2,0.00,Very Positive,1,1,2012-08-21,1,1,0,0,0,0,0,0,0,0,0,0
3,1091500,Cyberpunk 2077,59.99,Very Positive,1,0,2020-12-09,0,0,0,0,0,0,0,1,0,0,0,0
4,2488370,Cash Cleaner Simulator,14.79,Very Positive,1,0,2025-05-08,0,0,0,1,0,1,0,0,0,1,0,0


In [30]:
df.to_excel('processed_gamelist.xlsx')